# **ENCODER-DECODER VÀ BÀI TOÁN TÓM TẮT VĂN BẢN**


---


Model: ViT5-base + Synthetic Data + LLM Evaluation

Dataset: fcsn37/vietnamese-text-summarization-30k + Gemini synthetic



## **Giới thiệu:**
Encoder-Decoder và bài toán Tóm tắt văn bản:
- Fine-tune mô hình pretrained trên dữ liệu tự thu thập/lấy từ nguồn mở (dataset vietnamese-text-summarization từ Hugging Face).
- Áp dụng data augmentation bằng synthetic data sinh từ LLM mạnh (Gemini 2.5-flash) để tăng cường dataset.
- Đánh giá mô hình bằng metrics truyền thống (ROUGE) và LLM-based evaluation (sử dụng Gemini để chấm điểm coherence, relevance, v.v.).
- So sánh hiệu suất trước/sau augmentation và rút ra nhận xét về tính tổng quát của mô hình.

## **Thiết lập:**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### **Cài đặt thư viện:**

In [ ]:
! pip install -q transformers datasets sentencepiece rouge-score accelerate wandb google-generativeai openai underthesea tensorflow openai
! pip install evaluate underthesea

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.4/978.4 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.1 MB/s eta 0:00:00


In [ ]:
import torch
import os
import evaluate
import numpy as np
import pandas as pd
from openai import OpenAI
import google.generativeai as genai
import wandb
import re
import json
import kagglehub
import underthesea
import matplotlib.pyplot as plt
import time
import random
from datetime import datetime
from huggingface_hub import login, HfApi
from rouge_score import rouge_scorer
from tqdm import tqdm
from datasets import load_dataset, DatasetDict, concatenate_datasets, Dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments, Seq2SeqTrainer,
    DataCollatorForSeq2Seq, pipeline,
    TrainerCallback
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from underthesea import word_tokenize, text_normalize
from sklearn.model_selection import train_test_split
from google.colab import files

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

#### **Kết nối Hugging Face:**

*Truy cập link notebook (được cung cấp) để lấy api/tokens*

In [ ]:
# ! hf auth login

#### **Kết nối GenAI API:**

In [ ]:
GEMINI_API_KEY = "API_KEYS_PROVIDED"
genai.configure(api_key=GEMINI_API_KEY)

#### **Kết nối WANDB:**

In [ ]:
WANDB_PROJECT = "vit5-text-summarization"
WANDB_API_KEY = "WANDB_API_KEYS_PROVIDED"
wandb.login(key=WANDB_API_KEY)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fcsn_37 (fcsn_37-ton-duc-thang-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## **Chuẩn bị dữ liệu:**

1. Load dataset gốc từ Hugging Face ("fcsn37/vietnamese-text-summarization-30k") và stopwords từ file txt.
2. Tiền xử lý: Tokenize input (text) và target (summary) với max_length 1024/128, sử dụng ViT5 tokenizer.
3. Augmentation: Sử dụng Gemini để tạo synthetic data (2 variants per sample, thay đổi diễn đạt nhưng giữ ý nghĩa), xử lý rate limit bằng retry và sleep. Augment 200-1000 samples từ train split.
4. Kết hợp data: Concatenate dataset gốc với augmented, lưu thành dataset mới và push lên Hugging Face ("fcsn37/vietnamese-text-summarization-augmented_dataset").
5. Chia subset: Train (5000 samples), validation/test (200 samples each), shuffle với seed 42.

In [ ]:
data = pd.read_csv("hf://datasets/fcsn37/vietnamese-text-summarization/data_summary.csv")
data = data[['Text', 'Summary']]

In [ ]:
stopword = pd.read_csv("hf://datasets/fcsn37/vietnamese-stopwords/vietnamese-stopwords.txt", header=None, names=["stopword"])
print(stopword.head())

  stopword
0     a lô
1     a ha
2       ai
3    ai ai
4   ai nấy


### **Tiền xử lý dữ liệu:**

In [ ]:
print(data.shape)

(102681, 2)


In [ ]:
data = data.sample(n=30000, random_state=42).reset_index(drop=True)
print(data.shape)

(30000, 2)


In [ ]:
data.head(1)

,Text,Summary
0,"Hiện tại, Bitcoin được giao dịch quanh mốc 40.500 USD. Đêm qua, CEO Tesla Elon Musk chia sẻ trên Twitter rằng hãng sẽ chấp nhận thanh toán bằng Bitcoin trở lại khi những thợ đào sử dụng năng lượng sạch hợp lý. Trước tweet của Musk, giá đồng tiền số này chỉ dao động quanh 35.000 - 36.000 USD. CEO Tesla cũng phủ nhận cáo buộc thao túng thị trường tiền ảo. Tỷ phú Mỹ thông tin ""Tesla chỉ bán khoảng 10% số lượng Bitcoin nắm giữ"" để xác nhận đồng tiền này có tính thanh khoản dễ dàng không cần tác động đến thị trường. Cùng với động thái từ Elon Musk, đà tăng của Bitcoin cũng một phần nhờ phù thủy đầu tư Tudor Jones, nhà sáng lập quỹ quản lý tài sản Tudor Investment. ""Tôi thích Bitcoin như một công cụ để đa dạng hóa danh mục đầu tư. Mọi người hỏi nên làm gì với Bitcoin của mình. Tôi muốn danh mục 5% bằng vàng, 5% bằng Bitcoin, 5% bằng tiền mặt, 5% với hàng hóa"", Tudor Jones chia sẻ với CNBC. Ngay sau chia sẻ này, giá Bitcoin bật tăng 700 USD. Ông nói thêm xem Bitcoin như một câu chuyện về sự giàu có và bảo vệ sự giàu có theo thời gian. Tuần trước, đồng tiền số giá trị nhất thế giới cũng đón các thông tin tích cực như El Salvadol thông qua luật về Bitcoin và trở thành quốc gia công nhận nó như một đồng tiền hợp pháp. Tổng thống nước này tin rằng Bitcoin sẽ giúp thúc đẩy kinh tế, cải thiện tỷ lệ tiếp cận ngân hàng thấp của nước này và tạo điều kiện để chuyển kiều hối về nước mỗi năm nhanh hơn. Diễn biến tích cực của Bitcoin hôm nay cũng kéo thị trường tiền số cùng đi lên khi nhóm 10 đồng tiền số vốn hoá lớn nhất xanh trở lại sau 2 tuần trồi sụt. 24h qua, đồn Ether tăng hơn 8,5%, Dogecoin tăng 7,2%, Litecoin tăng 7,6%, theo Coinmarketcap. Tú Anh (theo CNBC)","Sau gần một ngày Musk nói Tesla có thể lại chấp nhận Bitcoin, đồng tiền này đã tăng gần 13%, lấy lại mốc 40.000 USD."


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


### **Loại bỏ phần tử rỗng:**

In [ ]:
data.drop_duplicates(subset=['Text'],inplace=True)
data.dropna(axis=0,inplace=True)

### **Chuẩn hoá văn bản:**

In [ ]:
def text_cleaner(text, num=0):
    # 1. Chuyển toàn bộ văn bản thành chữ thường
    newString = text.lower()
    # 2. Loại bỏ nội dung nằm trong dấu ngoặc tròn ( )
    newString = re.sub(r'\([^)]*\)', '', newString)
    # 3. Xóa dấu ngoặc kép "
    newString = newString.replace('"', '')
    # 4. Loại bỏ dấu câu và ký tự đặc biệt
    # Chỉ giữ lại chữ cái tiếng Việt, chữ số và khoảng trắng
    newString = re.sub(
        r"[^0-9a-zA-Záàảãạăắằẵặẳâấầậẫẩéèẻẽẹêếềểễệ"
        r"íìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữự"
        r"ýỳỷỹỵđ\s]",
        " ",
        newString
    )
    # 5. Chuẩn hóa khoảng trắng thừa
    newString = re.sub(r"\s+", " ", newString).strip()
    # 6. Tách từ bằng underthesea
    tokens = word_tokenize(newString, format="text").split()
    # 7. Loại bỏ stopword
    if num == 0:
        tokens = [w for w in tokens if w not in stopword]
    # 8. Loại bỏ những từ chỉ gồm 1 ký tự
    longwords = [w for w in tokens if len(w) > 1]
    return " ".join(longwords)

In [ ]:
cleaned_text = []
for t in data['Text']:
    cleaned_text.append(text_cleaner(t,0))

In [ ]:
cleaned_text[:1]

['hiện_tại bitcoin được giao_dịch quanh mốc 40 500 usd đêm qua ceo tesla elon musk chia_sẻ trên twitter rằng hãng sẽ chấp_nhận thanh_toán bằng bitcoin trở_lại khi những thợ đào sử_dụng năng_lượng sạch hợp_lý trước tweet của musk giá đồng_tiền số này chỉ dao_động quanh 35 000 36 000 usd ceo tesla cũng phủ_nhận cáo_buộc thao_túng thị_trường tiền ảo tỷ_phú_mỹ thông_tin tesla chỉ bán khoảng 10 số_lượng bitcoin nắm giữ để xác_nhận đồng_tiền này có tính thanh_khoản dễ_dàng không cần tác_động đến thị_trường cùng với động_thái từ elon musk đà tăng của bitcoin cũng một phần nhờ phù_thủy đầu_tư tudor jones nhà sáng_lập quỹ quản_lý tài_sản tudor investment tôi thích bitcoin như một công_cụ để đa_dạng hóa danh_mục đầu_tư mọi người hỏi nên làm gì với bitcoin của mình tôi muốn danh_mục bằng vàng bằng bitcoin bằng tiền_mặt với hàng hóa tudor jones chia_sẻ với cnbc ngay sau chia_sẻ này giá bitcoin bật tăng 700 usd ông nói thêm xem bitcoin như một câu_chuyện về sự giàu_có và bảo_vệ sự giàu_có theo thời

In [ ]:
cleaned_summary = []
for t in data['Summary']:
    cleaned_summary.append(text_cleaner(t,1))

In [ ]:
cleaned_summary[:1]

['sau gần một ngày musk nói tesla có_thể lại chấp_nhận bitcoin đồng_tiền này đã tăng gần 13 lấy lại mốc 40 000 usd']

In [ ]:
data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary

In [ ]:
data.head(1)

,Text,Summary,cleaned_text,cleaned_summary
0,"Hiện tại, Bitcoin được giao dịch quanh mốc 40.500 USD. Đêm qua, CEO Tesla Elon Musk chia sẻ trên Twitter rằng hãng sẽ chấp nhận thanh toán bằng Bitcoin trở lại khi những thợ đào sử dụng năng lượng sạch hợp lý. Trước tweet của Musk, giá đồng tiền số này chỉ dao động quanh 35.000 - 36.000 USD. CEO Tesla cũng phủ nhận cáo buộc thao túng thị trường tiền ảo. Tỷ phú Mỹ thông tin ""Tesla chỉ bán khoảng 10% số lượng Bitcoin nắm giữ"" để xác nhận đồng tiền này có tính thanh khoản dễ dàng không cần tác động đến thị trường. Cùng với động thái từ Elon Musk, đà tăng của Bitcoin cũng một phần nhờ phù thủy đầu tư Tudor Jones, nhà sáng lập quỹ quản lý tài sản Tudor Investment. ""Tôi thích Bitcoin như một công cụ để đa dạng hóa danh mục đầu tư. Mọi người hỏi nên làm gì với Bitcoin của mình. Tôi muốn danh mục 5% bằng vàng, 5% bằng Bitcoin, 5% bằng tiền mặt, 5% với hàng hóa"", Tudor Jones chia sẻ với CNBC. Ngay sau chia sẻ này, giá Bitcoin bật tăng 700 USD. Ông nói thêm xem Bitcoin như một câu chuyện về sự giàu có và bảo vệ sự giàu có theo thời gian. Tuần trước, đồng tiền số giá trị nhất thế giới cũng đón các thông tin tích cực như El Salvadol thông qua luật về Bitcoin và trở thành quốc gia công nhận nó như một đồng tiền hợp pháp. Tổng thống nước này tin rằng Bitcoin sẽ giúp thúc đẩy kinh tế, cải thiện tỷ lệ tiếp cận ngân hàng thấp của nước này và tạo điều kiện để chuyển kiều hối về nước mỗi năm nhanh hơn. Diễn biến tích cực của Bitcoin hôm nay cũng kéo thị trường tiền số cùng đi lên khi nhóm 10 đồng tiền số vốn hoá lớn nhất xanh trở lại sau 2 tuần trồi sụt. 24h qua, đồn Ether tăng hơn 8,5%, Dogecoin tăng 7,2%, Litecoin tăng 7,6%, theo Coinmarketcap. Tú Anh (theo CNBC)","Sau gần một ngày Musk nói Tesla có thể lại chấp nhận Bitcoin, đồng tiền này đã tăng gần 13%, lấy lại mốc 40.000 USD.",hiện_tại bitcoin được giao_dịch quanh mốc 40 500 usd đêm qua ceo tesla elon musk chia_sẻ trên twitter rằng hãng sẽ chấp_nhận thanh_toán bằng bitcoin trở_lại khi những thợ đào sử_dụng năng_lượng sạch hợp_lý trước tweet của musk giá đồng_tiền số này chỉ dao_động quanh 35 000 36 000 usd ceo tesla cũng phủ_nhận cáo_buộc thao_túng thị_trường tiền ảo tỷ_phú_mỹ thông_tin tesla chỉ bán khoảng 10 số_lượng bitcoin nắm giữ để xác_nhận đồng_tiền này có tính thanh_khoản dễ_dàng không cần tác_động đến thị_trường cùng với động_thái từ elon musk đà tăng của bitcoin cũng một phần nhờ phù_thủy đầu_tư tudor jones nhà sáng_lập quỹ quản_lý tài_sản tudor investment tôi thích bitcoin như một công_cụ để đa_dạng hóa danh_mục đầu_tư mọi người hỏi nên làm gì với bitcoin của mình tôi muốn danh_mục bằng vàng bằng bitcoin bằng tiền_mặt với hàng hóa tudor jones chia_sẻ với cnbc ngay sau chia_sẻ này giá bitcoin bật tăng 700 usd ông nói thêm xem bitcoin như một câu_chuyện về sự giàu_có và bảo_vệ sự giàu_có theo thời_gian tuần trước đồng_tiền số giá_trị nhất thế_giới cũng đón các thông_tin tích_cực như el_salvadol thông_qua luật về bitcoin và trở_thành quốc_gia công_nhận nó như một đồng_tiền hợp_pháp tổng_thống nước này tin rằng bitcoin sẽ giúp thúc_đẩy kinh_tế cải_thiện tỷ_lệ tiếp_cận ngân_hàng thấp của nước này và tạo điều_kiện để chuyển kiều_hối về nước mỗi năm nhanh hơn diễn_biến tích_cực của bitcoin hôm_nay cũng kéo thị_trường tiền số cùng đi lên khi nhóm 10 đồng_tiền số vốn hóa lớn nhất xanh trở_lại sau tuần trồi sụt 24 qua đồn ether tăng hơn dogecoin tăng litecoin tăng theo coinmarketcap tú anh,sau gần một ngày musk nói tesla có_thể lại chấp_nhận bitcoin đồng_tiền này đã tăng gần 13 lấy lại mốc 40 000 usd


### **Lưu lại dữ liệu đã được xử lý**

In [ ]:
dataset = data[['cleaned_text', 'cleaned_summary']].copy()

dataset = dataset.rename(columns={
    'cleaned_text': 'text',
    'cleaned_summary': 'summary'
})

hf_dataset = Dataset.from_pandas(dataset, preserve_index=False)
dataset_dict = hf_dataset.train_test_split(test_size=0.2, seed=42)
dataset_dict = DatasetDict({
    'train': dataset_dict['train'],
    'validation': dataset_dict['test'].train_test_split(test_size=0.5, seed=42)['train'],
    'test': dataset_dict['test'].train_test_split(test_size=0.5, seed=42)['test']
})
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 23435
    })
    validation: Dataset({
        features: ['text', 'summary'],
        num_rows: 2929
    })
    test: Dataset({
        features: ['text', 'summary'],
        num_rows: 2930
    })
})

In [ ]:
login(token="API_KEYS_PROVIDED")

In [ ]:
# ! hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `en

In [ ]:
# dataset_dict.push_to_hub("fcsn37/vietnamese-text-summarization-30k")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/24 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              :   1%|1         |  525kB / 51.4MB            

...

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              :  59%|#####9    | 3.67MB / 6.22MB            

CommitInfo(commit_url='https://huggingface.co/datasets/fcsn37/vietnamese-text-summarization-30k/commit/8cac5ef32907722849387378511b96894350cb98', commit_message='Upload dataset', commit_description='', oid='8cac5ef32907722849387378511b96894350cb98', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/fcsn37/vietnamese-text-summarization-30k', endpoint='https://huggingface.co', repo_type='dataset', repo_id='fcsn37/vietnamese-text-summarization-30k'), pr_revision=None, pr_num=None)

### **Tải dữ liệu đã xử lý từ Hugging Face:**

In [ ]:
dataset = load_dataset("fcsn37/vietnamese-text-summarization-30k")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/565 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/51.4M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/6.49M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/6.22M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/23435 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2929 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2930 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")
model     = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/904M [00:00<?, ?B/s]

In [ ]:
MAX_INPUT_LENGTH  = 1024
MAX_TARGET_LENGTH = 128

def preprocess_function(examples):
    inputs  = examples["text"]
    targets = examples["summary"]

    model_inputs = tokenizer(
        inputs,
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
        padding=False,
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=MAX_TARGET_LENGTH,
            truncation=True,
            padding=False,
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

Map:   0%|          | 0/23435 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4118: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2929 [00:00<?, ? examples/s]

Map:   0%|          | 0/2930 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 23435
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2929
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2930
    })
})


### **Dữ liệu tổng hợp:**

In [ ]:
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash

In [ ]:
gemini_model = genai.GenerativeModel('gemini-2.5-flash')

In [ ]:
generation_config = {
    "temperature": 0.7,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 2048,
}

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"}
]

In [ ]:
def generate_summary_variants(text, summary, num_variants=2, max_retries=3):
    """Tạo các biến thể tóm tắt cho văn bản gốc"""
    prompt = f"""
Bạn là chuyên gia về tóm tắt văn bản tiếng Việt. Tạo {num_variants} biến thể tóm tắt KHÁC NHAU, giữ nguyên ý nghĩa nhưng thay đổi cách diễn đạt.

VĂN BẢN GỐC:
{text}

TÓM TẮT GỐC:
{summary}

YÊU CẦU:
1. Giữ nguyên nội dung chính
2. Thay đổi cách diễn đạt, cấu trúc câu
3. Độ dài tương đương (±20%)
4. Tiếng Việt chuẩn, tự nhiên
5. Trả về JSON: {{"variants": [{{"summary": "..."}}]}}

Tạo {num_variants} biến thể:
"""

    for attempt in range(max_retries):
        try:
            response = gemini_model.generate_content(
                prompt,
                generation_config=generation_config,
                safety_settings=safety_settings
            )

            response_text = response.text.strip()

            if "```json" in response_text:
                json_str = response_text.split("```json")[1].split("```")[0].strip()
            elif "```" in response_text:
                json_str = response_text.split("```")[1].split("```")[0].strip()
            else:
                json_str = response_text

            result = json.loads(json_str)
            variants = []

            for variant in result.get("variants", []):
                variants.append({
                    "text": text,
                    "summary": variant.get("summary", "")
                })

            return variants

        except Exception as e:
            error_msg = str(e)

            # Xử lý rate limit
            if "429" in error_msg or "quota" in error_msg.lower():
                if attempt < max_retries - 1:
                    # Exponential backoff: 60s, 120s, 240s
                    wait_time = 60 * (2 ** attempt)
                    print(f"Rate limit! Chờ {wait_time}s...")
                    time.sleep(wait_time)
                    continue
                else:
                    print(f"Đã hết retry. Bỏ qua mẫu này.")
                    return []
            else:
                print(f"Lỗi: {error_msg}")
                return []

    return []

def augment_dataset(dataset_dict, split='train', num_samples=1000, variants_per_sample=2, delay=2.0):
    """Augment dataset với Gemini - Tự động xử lý rate limit"""
    augmented_data = []

    # Lấy split cụ thể từ DatasetDict
    dataset = dataset_dict[split]

    # Shuffle và lấy mẫu
    dataset_shuffled = dataset.shuffle(seed=42)
    num_samples = min(num_samples, len(dataset))
    samples = dataset_shuffled.select(range(num_samples))

    success_count = 0
    fail_count = 0
    request_count = 0

    for idx, item in enumerate(tqdm(samples, desc="Augmenting")):
        try:
            variants = generate_summary_variants(
                item['text'],
                item['summary'],
                num_variants=variants_per_sample
            )

            if variants:
                augmented_data.extend(variants)
                success_count += 1
                request_count += 1
            else:
                fail_count += 1

            # Sau mỗi 190 requests, chờ 70 giây để tránh vượt quota
            if request_count > 0 and request_count % 190 == 0:
                remaining = num_samples - idx - 1
                if remaining > 0:
                    print(f"\nĐã xử lý {request_count} requests. Chờ 70s để tránh rate limit...")
                    time.sleep(70)

            time.sleep(delay)

        except KeyboardInterrupt:
            print("\nDừng bởi người dùng!")
            break
        except:
            fail_count += 1
            continue

    print(f"\nThành công: {success_count}/{num_samples} |Thất bại: {fail_count}")
    return augmented_data

In [ ]:
# augmented_list = augment_dataset(
#     dataset_dict=dataset,
#     split='train',
#     num_samples=200,
#     variants_per_sample=2,
#     delay=2.0
# )

Augmenting:   0%|          | 1/200 [00:10<33:36, 10.13s/it]

Lỗi: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. The candidate's [finish_reason](https://ai.google.dev/api/generate-content#finishreason) is 2.


Augmenting:   1%|          | 2/200 [00:23<39:08, 11.86s/it]

Lỗi: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. The candidate's [finish_reason](https://ai.google.dev/api/generate-content#finishreason) is 2.


Augmenting:   2%|▎         | 5/200 [00:56<35:26, 10.91s/it]

...

Đã hết retry. Bỏ qua mẫu này.


Augmenting:  94%|█████████▍| 188/200 [50:03<31:01, 155.13s/it]WARNING:tornado.access:429 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 456.84ms


Rate limit! Chờ 60s...


Augmenting:  94%|█████████▍| 188/200 [51:01<03:15, 16.28s/it] 


Dừng bởi người dùng!

Thành công: 97/200 |Thất bại: 91


### **Lưu lại dữ liệu đã được tổng hợp:**

In [ ]:
# Chuyển thành DataFrame
augmented_df = pd.DataFrame(augmented_list)

print(f"Dataset gốc (train): {len(dataset['train'])} mẫu")
print(f"Dữ liệu augmented: {len(augmented_df)} mẫu")

Dataset gốc (train): 23435 mẫu
Dữ liệu augmented: 194 mẫu


In [ ]:
augmented_dataset = Dataset.from_pandas(augmented_df)
# dataset['train'] = concatenate_datasets([dataset['train'], augmented_dataset])

print(f"Dataset train sau augment: {len(dataset['train'])} mẫu")

Dataset train sau augment: 23629 mẫu


In [ ]:
print(augmented_dataset)

Dataset({
    features: ['text', 'summary'],
    num_rows: 194
})


In [ ]:
# augmented_dataset.push_to_hub("fcsn37/vietnamese-text-summarization-augmented_dataset")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              : 100%|##########|  222kB /  222kB            

CommitInfo(commit_url='https://huggingface.co/datasets/fcsn37/vietnamese-text-summarization-augmented_dataset/commit/1fc201926dfe4723587fc79afefd8ae0d94f51e0', commit_message='Upload dataset', commit_description='', oid='1fc201926dfe4723587fc79afefd8ae0d94f51e0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/fcsn37/vietnamese-text-summarization-augmented_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='fcsn37/vietnamese-text-summarization-augmented_dataset'), pr_revision=None, pr_num=None)

## **Huấn luyện mô hình:**

1. Define hyperparameters: Batch size 4, gradient accumulation 4, LR 3e-5, epochs 3, warmup 100, max_input 256, max_target 64.
2. Sử dụng DataCollatorForSeq2Seq và compute_metrics với ROUGE.
3. Huấn luyện baseline trên data gốc.
4. Huấn luyện augmented trên data kết hợp (gốc + 193 augmented samples).
5. Sử dụng Seq2SeqTrainer với callbacks để push checkpoint lên HF repo.
6. Lưu model final và evaluate trên validation.

In [ ]:
MODEL_NAME = "VietAI/vit5-base"
MAX_INPUT_LENGTH = 256
MAX_TARGET_LENGTH = 64

BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = 3e-5
NUM_EPOCHS = 3
WARMUP_STEPS = 100

HF_TOKEN = "HF_TOKENS_PROVIDED"
DATASET_NAME = "fcsn37/vietnamese-text-summarization-30k"
AUGMENTED_DATASET = "fcsn37/vietnamese-text-summarization-augmented_dataset"

WANDB_PROJECT = "vit5-text-summarization"
WANDB_API_KEY = "API_KEYS_PROVIDED"

In [ ]:
dataset = load_dataset(DATASET_NAME)

train_size = 5000
dataset['train'] = dataset['train'].shuffle(seed=42).select(range(train_size))
dataset['validation'] = dataset['validation'].shuffle(seed=42).select(range(200))
dataset['test'] = dataset['test'].shuffle(seed=42).select(range(200))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_function(examples):
    inputs = examples["text"]
    targets = examples["summary"]

    model_inputs = tokenizer(
        inputs,
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
        padding=False,
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=MAX_TARGET_LENGTH,
            truncation=True,
            padding=False,
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=None,
    padding=True
)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4118: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )

    return {k: round(v * 100, 2) for k, v in result.items()}

In [ ]:
class SafePushCallback(TrainerCallback):
    def __init__(self, tokenizer, repo_name, hf_token, experiment_name):
        self.tokenizer = tokenizer
        self.repo_name = repo_name
        self.api = HfApi(token=hf_token)
        self.experiment_name = experiment_name

    def on_epoch_end(self, args, state, control, model=None, **kwargs):
        epoch = int(state.epoch)
        checkpoint_name = f"{self.experiment_name}-epoch-{epoch}"
        checkpoint_dir = f"{args.output_dir}/{checkpoint_name}"

        os.makedirs(checkpoint_dir, exist_ok=True)
        model.save_pretrained(checkpoint_dir)
        self.tokenizer.save_pretrained(checkpoint_dir)

        try:
            self.api.upload_folder(
                folder_path=checkpoint_dir,
                repo_id=self.repo_name,
                path_in_repo=checkpoint_name,
                repo_type="model",
                commit_message=f"Checkpoint {checkpoint_name}"
            )
        except Exception as e:
            pass

        return control

In [ ]:
def train_model(experiment_name, train_dataset, eval_dataset, repo_name):

    torch.cuda.empty_cache()

    output_dir = f"./results/{experiment_name}"

    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
    model.gradient_checkpointing_enable()

    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model,
        padding=True
    )

    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=LEARNING_RATE,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        weight_decay=0.01,
        save_total_limit=2,
        num_train_epochs=NUM_EPOCHS,
        predict_with_generate=True,
        fp16=torch.cuda.is_available(),
        logging_steps=50,
        warmup_steps=WARMUP_STEPS,
        load_best_model_at_end=True,
        metric_for_best_model="rouge1",
        greater_is_better=True,
        report_to="wandb",
        run_name=experiment_name,
        push_to_hub=False,
        label_smoothing_factor=0.1,
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        processing_class=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[SafePushCallback(tokenizer, repo_name, HF_TOKEN, experiment_name)]
    )

    if os.path.exists(output_dir):
        checkpoints = [d for d in os.listdir(output_dir) if d.startswith(experiment_name)]
        if checkpoints:
            latest = max(checkpoints, key=lambda x: os.path.getmtime(os.path.join(output_dir, x)))
            trainer.train(resume_from_checkpoint=os.path.join(output_dir, latest))
        else:
            trainer.train()
    else:
        trainer.train()

    final_dir = f"{output_dir}/final"
    trainer.save_model(final_dir)
    tokenizer.save_pretrained(final_dir)

    try:
        api = HfApi(token=HF_TOKEN)
        api.upload_folder(
            folder_path=final_dir,
            repo_id=repo_name,
            path_in_repo=f"{experiment_name}-final",
            repo_type="model"
        )
    except Exception as e:
        pass

    eval_results = trainer.evaluate()

    return eval_results, trainer

### **Huấn luyện trên tập dữ liệu nguyên mẫu:**

In [ ]:
baseline_results, baseline_trainer = train_model(
    experiment_name="baseline",
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    repo_name="fcsn37/vit5-summarization-baseline"
)

print("\nKet qua Baseline:")
print(pd.DataFrame([baseline_results]))

wandb.finish()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,3.870900,3.661714,36.250000,12.610000,25.530000,25.590000
2,3.644800,3.553145,34.910000,13.480000,24.980000,25.010000
3,3.461800,3.530811,34.950000,13.380000,25.380000,25.380000


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...line-epoch-1/spiece.model: 100%|##########|  820kB /  820kB            

  ...epoch-1/model.safetensors:   0%|          |  555kB /  904MB            

...

  ...seline/final/spiece.model: 100%|##########|  820kB /  820kB            

  ...e/final/model.safetensors:   4%|3         | 33.5MB /  904MB            

  ...e/final/training_args.bin: 100%|##########| 6.03kB / 6.03kB            


Ket qua Baseline:
   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0   3.661714        36.25        12.61        25.53           25.59   

   eval_runtime  eval_samples_per_second  eval_steps_per_second  epoch  
0         29.86                    6.698                  1.674    3.0  


eval/loss,█▂▁█
eval/rouge1,█▁▁█
eval/rouge2,▁█▇▁
eval/rougeL,█▁▆█
eval/rougeLsum,█▁▅█
eval/runtime,▅▅█▁
eval/samples_per_second,▄▄▁█
eval/steps_per_second,▄▄▁█
train/epoch,▂▃▁▁▂▂▃▃▃▃▄▄▅▅▅▆▆▆▇▇▇████
train/global_step,▁▁▁▁▂▂▃▃▃▃▄▄▅▅▅▆▆▆▇▇▇████
+3,...


### **Huấn luyện trên tập dữ liệu kết hợp:**

In [ ]:
augmented_dataset = load_dataset(AUGMENTED_DATASET, split='train')

num_augmented = min(len(augmented_dataset), 500)
augmented_sample = augmented_dataset.shuffle(seed=42).select(range(num_augmented))

combined_train = concatenate_datasets([
    dataset['train'],
    augmented_sample
])

combined_tokenized = combined_train.map(
    preprocess_function,
    batched=True,
    remove_columns=combined_train.column_names
)

print(f"Du lieu training: {len(combined_tokenized)} mau")

augmented_results, augmented_trainer = train_model(
    experiment_name="augmented",
    train_dataset=combined_tokenized,
    eval_dataset=tokenized_datasets["validation"],
    repo_name="fcsn37/vit5-summarization-augmented"
)

print("\nKet qua Augmented:")
print(pd.DataFrame([augmented_results]))

wandb.finish()

Map:   0%|          | 0/5194 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4118: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Du lieu training: 5194 mau


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,3.896400,3.642673,35.360000,12.680000,24.790000,24.870000
2,3.643800,3.548826,35.150000,13.560000,24.960000,24.960000
3,3.478500,3.526063,36.220000,13.960000,25.690000,25.750000


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].



Ket qua Augmented:
   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0   3.526063        36.22        13.96        25.69           25.75   

   eval_runtime  eval_samples_per_second  eval_steps_per_second  epoch  
0       30.1081                    6.643                  1.661    3.0  


eval/loss,█▂▁▁
eval/rouge1,▂▁██
eval/rouge2,▁▆██
eval/rougeL,▁▂██
eval/rougeLsum,▁▂██
eval/runtime,█▆▁▁
eval/samples_per_second,▁▃██
eval/steps_per_second,▁▃██
train/epoch,▁▁▂▂▃▃▃▃▄▄▄▅▅▆▆▆▆▇▇▇████
train/global_step,▁▁▂▂▃▃▃▃▄▄▄▅▅▆▆▆▆▇▇▇████
+3,...


### **So sánh kết quả trước và sau khi tổng hợp:**

In [ ]:
comparison_data = {
    'Model': ['Baseline'],
    'ROUGE-1': [baseline_results.get('eval_rouge1', 0)],
    'ROUGE-2': [baseline_results.get('eval_rouge2', 0)],
    'ROUGE-L': [baseline_results.get('eval_rougeL', 0)],
}

if augmented_results:
    comparison_data['Model'].append('Augmented')
    comparison_data['ROUGE-1'].append(augmented_results.get('eval_rouge1', 0))
    comparison_data['ROUGE-2'].append(augmented_results.get('eval_rouge2', 0))
    comparison_data['ROUGE-L'].append(augmented_results.get('eval_rougeL', 0))

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df)

       Model  ROUGE-1  ROUGE-2  ROUGE-L
0   Baseline    36.25    12.61    25.53
1  Augmented    36.22    13.96    25.69


In [ ]:
def generate_summary(model, tokenizer, text, max_length=64):
    inputs = tokenizer(text, return_tensors="pt", max_length=MAX_INPUT_LENGTH, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        num_beams=4,
        early_stopping=True
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

baseline_trainer.model.to(device)

for i in range(3):
    sample = dataset['test'][i]
    text = sample['text']
    original_summary = sample['summary']
    predicted_summary = generate_summary(baseline_trainer.model, tokenizer, text)

    print(f"\nVi du {i+1}:")
    print(f"Van ban: {text}...")
    print(f"Tom tat goc: {original_summary}")
    print(f"Tom tat du doan: {predicted_summary}")
    print("-" * 80)


Vi du 1:
Van ban: phó tổng_thống mỹ_kamala harris mặc_dù tôi có_thể là người phụ_nữ đầu_tiên giữ vị_trí này nhưng tôi sẽ không phải là người cuối_cùng bà kamala harris từng nhấn_mạnh trong một bài phát_biểu hồi tháng 11 2020 sau khi liên_danh tranh_cử joe biden kamala_harris giành chiến_thắng trong cuộc bầu_cử tổng_thống mỹ với lễ tuyên_thệ nhậm_chức vào ngày 20 2021 nữ cựu thượng_nghị_sĩ bang california đã làm_nên lịch_sử với tư_cách là người phụ_nữ đầu_tiên và cũng là người phụ_nữ da màu đầu_tiên trở_thành phó tổng_thống mỹ bà harris sinh ngày 20 10 1964 với tên đầy_đủ là kamala devi harris là con của những người nhập_cư đến mỹ mẹ bà shyamala gopalan harris là người nhập_cư từ ấn_độ còn cha bà ông donald haris là một giáo_sư kinh_tế người mỹ gốc jamaica bà harris lớn lên với sự ảnh_hưởng sâu_sắc của người mẹ gốc ấn và nền_tảng tư_duy từ cha mình trong suốt quá_trình tranh_cử bà harris thường_xuyên đề_cập đến việc các hoạt_động đấu_tranh vì nhân_quyền của cha_mẹ đã ảnh_hưởng tới bà n